* Selectionner parmi les tables les colonnes qui de près ou de loin pourraient être intéressantes pour l'étude (d02_intermediate)

* Rechercher s'il existe des valeurs manquantes, les traiter (d02_intermediate)

* Rechercher s'il y a des colonnes qu'il faudrait transformer en datetime (d02_intermediate)

* Vérifier la consistence des données (d02_intermediate)

* Rechercher les valeurs abbérantes et les traiter (d02_intermediate)


# Nettoyage des données

## 0. Set up

### Mise en place de l'environnement

In [1]:
#Importation des Modules nécessaires
import pandas as pd
import sqlalchemy
import mysql.connector


#Pour que python reconnaisse un dossier comme un module il faut ajouter en son sein un fichier __init__.py 
#et également l'ajouter au PATH

import sys
sys.path.insert(0, "/home/apprenant/simplon_projects/american_dream/")


#On peut ensuite importer le dossier comme un module

from src.d00_utils.mysql_utils import mysql_connect, save_to_mysql
connect = mysql_connect()

### Importation des données depuis MySQL

In [2]:

df1 = pd.read_sql("excel_table",con=connect)
dfk = pd.read_sql("csv_table",con=connect)



In [3]:
print(df1.duplicated().value_counts())

False    8627
dtype: int64


## 1. Sélection des colonnes



J'affiche un échantillon de mes données pour analyser la pertinence des différentes colonnes


In [4]:


print(df1.head())
print(df1.columns)



   index  Survey Year           Timestamp  SalaryUSD        Country  \
0      0         2020 2020-01-04 18:50:34   115000.0  United States   
1      1         2020 2020-01-04 10:43:02   100000.0  United States   
2      2         2020 2020-01-04 09:51:46   100000.0          Spain   
3      3         2020 2020-01-04 01:08:54    70000.0  United States   
4      4         2020 2020-01-03 15:28:54   110000.0  United States   

  PostalCode       PrimaryDatabase  YearsWithThisDatabase  \
0         03  Microsoft SQL Server                     15   
1       None                 Other                      6   
2      28046  Microsoft SQL Server                      2   
3      94133  Microsoft SQL Server                      3   
4      95354                Oracle                     30   

                                      OtherDatabases    EmploymentStatus  ...  \
0  Microsoft SQL Server, MongoDB, Azure SQL DB (a...  Full time employee  ...   
1                                      MySQL


Je choisis uniquement les colonnes qui pourraient m'être utiles pour mon étude


In [5]:
df1 = df1[['SalaryUSD','Country','PostalCode','EmploymentStatus','JobTitle','ManageStaff','YearsWithThisTypeOfJob',
          'HowManyCompanies','OtherPeopleOnYourTeam', 'CompanyEmployeesOverall',
          'PopulationOfLargestCityWithin20Miles','EmploymentSector', 'LookingForAnotherJob', 'CareerPlansThisYear',
          'Gender', 'OtherJobDuties']]

Je récupère la liste des pays, pour vérifier qu'il n'y a pas de doublons.

In [6]:
countries = df1['Country'].unique()
countries.sort()
countries


array(['Albania', 'Anguilla', 'Argentina', 'Armenia', 'Australia',
       'Austria', 'Bahrain', 'Belarus', 'Belgium', 'Bermuda', 'Bolivia',
       'Brazil', 'Bulgaria', 'Canada', 'Cayman Islands', 'China',
       'Colombia', 'Costa Rica', 'Croatia', 'Czech Republic', 'Denmark',
       'Dominican Republic', 'Ecuador', 'El Salvador', 'Estonia',
       'Finland', 'France', 'Georgia', 'Germany', 'Ghana', 'Greece',
       'Guatemala', 'Guernsey', 'Hong Kong', 'Hungary', 'Iceland',
       'India', 'Indonesia', 'Iran', 'Ireland', 'Israel', 'Italy',
       'Jamaica', 'Jersey', 'Jordan', 'Kenya', 'Latvia', 'Lithuania',
       'Luxembourg', 'Macedonia', 'Malaysia', 'Maldives', 'Malta',
       'Mexico', 'Moldova', 'Nepal', 'Netherlands', 'New Zealand',
       'Nicaragua', 'Norway', 'Pakistan', 'Paraguay', 'Peru',
       'Philippines', 'Poland', 'Portugal', 'Puerto Rico', 'Qatar',
       'Romania', 'Russia', 'Saudi Arabia', 'Serbia and Montenegro',
       'Singapore', 'Slovakia', 'Slovenia', 'Sout

Il n'y a pas de doublons dans la liste des pays, je peux donc récupérer uniquement les lignes où la colonne Country est égale à 'United States' puisque c'est ce qui m'intéresse.

In [7]:
df1 = df1.loc[df1['Country'] == "United States"]
df1.head()

,SalaryUSD,Country,PostalCode,EmploymentStatus,JobTitle,ManageStaff,YearsWithThisTypeOfJob,HowManyCompanies,OtherPeopleOnYourTeam,CompanyEmployeesOverall,PopulationOfLargestCityWithin20Miles,EmploymentSector,LookingForAnotherJob,CareerPlansThisYear,Gender,OtherJobDuties
0,115000.0,United States,03,Full time employee,DBA (Production Focus - build & troubleshoot s...,No,5,1 (this is the only company where I've had thi...,2,Not Asked,20K-99K (large town),Private business,Not Asked,"Stay with the same employer, same role",Male,Not Asked
1,100000.0,United States,None,Full time employee,"Developer: App code (C#, JS, etc)",No,6,1 (this is the only company where I've had thi...,More than 5,Not Asked,1M+ (metropolis),Private business,Not Asked,Prefer not to say,Male,Not Asked
3,70000.0,United States,94133,Full time employee,Analyst,No,3,1 (this is the only company where I've had thi...,2,Not Asked,1M+ (metropolis),Private business,Not Asked,Change both employers and roles,Male,Not Asked
4,110000.0,United States,95354,Full time employee,DBA (General - splits time evenly between writ...,No,25,3,None,Not Asked,100K-299K (city),Non-profit,Not Asked,"Stay with the same employer, same role",Male,Not Asked
5,79500.0,United States,78727,Full time employee,Analyst,No,2,1 (this is the only company where I've had thi...,None,Not Asked,1M+ (metropolis),Private business,Not Asked,"Stay with the same employer, same role",Male,Not Asked




Je fais la même chose pour la seconde table


In [8]:
print(dfk.head())
print(dfk.columns)

   index  Unnamed: 0                                          Job Title  \
0      0           0  Data Analyst, Center on Immigration and Justic...   
1      1           1                               Quality Data Analyst   
2      2           2  Senior Data Analyst, Insights & Analytics Team...   
3      3           3                                       Data Analyst   
4      4           4                             Reporting Data Analyst   

              Salary Estimate  \
0  $37K-$66K (Glassdoor est.)   
1  $37K-$66K (Glassdoor est.)   
2  $37K-$66K (Glassdoor est.)   
3  $37K-$66K (Glassdoor est.)   
4  $37K-$66K (Glassdoor est.)   

                                     Job Description  Rating  \
0  Are you eager to roll up your sleeves and harn...     3.2   
1  Overview\n\nProvides analytical and technical ...     3.8   
2  We’re looking for a Senior Data Analyst who ha...     3.4   
3  Requisition NumberRR-0001939\nRemote:Yes\nWe c...     4.1   
4  ABOUT FANDUEL GROUP\n\nFanD

In [9]:
dfk = dfk[['Job Title', 'Salary Estimate',
       'Job Description', 'Company Name', 'Location', 'Headquarters',
       'Size', 'Founded', 'Type of ownership', 'Industry', 'Sector', 'Revenue'
]]


## 2. Valeurs manquantes
# 
Recherche des valeurs manquantes


In [10]:

print(df1.isnull().sum()) 
print(df1.shape)

SalaryUSD                                 0
Country                                   0
PostalCode                              466
EmploymentStatus                          0
JobTitle                                  0
ManageStaff                               0
YearsWithThisTypeOfJob                    0
HowManyCompanies                          0
OtherPeopleOnYourTeam                     0
CompanyEmployeesOverall                   0
PopulationOfLargestCityWithin20Miles      0
EmploymentSector                          0
LookingForAnotherJob                      0
CareerPlansThisYear                       0
Gender                                    0
OtherJobDuties                          385
dtype: int64
(5680, 16)




Il me manque 1390 veleurs pour PostalCode et 620 pour OtherJobDUties sur un total de 8627 lignes. Ces deux colonnes ne sont pas au centre de mon études donc ça se vaut pas le cout de supprimer toute la ligne pour elles

(Après sélection des valeurs pour USA uniquement: il manque 466 valeurs dans PostalCode (environ 8%) et 385 dans OtherJobDUties (moins de 7%))


### traitement de Postal code


Cela ne ferait pas de sens de remplacer le code postal par un code postal arbitraire


### Traitement de OtherJobDuties

In [11]:

print(df1.OtherJobDuties.unique())
print(df1.OtherJobDuties.value_counts())



), Developer: T-SQL'
 'Data Scientist, Developer: App code (C#, JS, etc), Developer: T-SQL, Engineer'
 'Architect, DBA (Development Focus - tunes queries, indexes, does deployments), Developer: App code (C#, JS, etc), Developer: T-SQL'
 'Analyst, Architect, DBA (Development Focus - tunes queries, indexes, does deployments), DBA (Production Focus - build & troubleshoot servers, HA/DR), Developer: Business Intelligence (SSRS, PowerBI, etc), Engineer'
 'Analyst, Architect, Data Scientist, Manager'
 'Analyst, Architect, DBA (General - splits time evenly between writing & tuning queries AND building & troubleshooting servers), Developer: Business Intelligence (SSRS, PowerBI, etc), Developer: T-SQL, Manager'
 'Architect, DBA (Production Focus - build & troubleshoot servers, HA/DR), Developer: Business Intelligence (SSRS, PowerBI, etc), Developer: T-SQL'
 'Analyst, DBA (General - splits time evenly between writing & tuning queries AND building & troubleshooting servers), Developer: App code (



Comme il y a trop de valeurs uniques différentes pour OtherJobDuties, je ne peux remplacer les valeurs manquantes par une valeur standard


### Conclusion sur les valeurs manquantes


Je choisis donc de conserver l'ensemble des données dans ma base mais je note un point de vigilance quand j'aurai à utiliser les variables codePostal et OtherJobDuties


### 2ème table

In [12]:
print(dfk.isnull().sum())
print(dfk.shape)

Job Title            0
Salary Estimate      0
Job Description      0
Company Name         1
Location             0
Headquarters         0
Size                 0
Founded              0
Type of ownership    0
Industry             0
Sector               0
Revenue              0
dtype: int64
(2253, 12)


Il n'y a qu'une seule valeur manquante dans 'Company Name' qui n'est pas un élément central, je décide donc de ne pas supprimer la ligne correspondante.

## 3. Traitement des dates


In [13]:
print(df1.dtypes)

SalaryUSD                               float64
Country                                  object
PostalCode                               object
EmploymentStatus                         object
JobTitle                                 object
ManageStaff                              object
YearsWithThisTypeOfJob                    int64
HowManyCompanies                         object
OtherPeopleOnYourTeam                    object
CompanyEmployeesOverall                  object
PopulationOfLargestCityWithin20Miles     object
EmploymentSector                         object
LookingForAnotherJob                     object
CareerPlansThisYear                      object
Gender                                   object
OtherJobDuties                           object
dtype: object


In [14]:
print(dfk.dtypes)

Job Title            object
Salary Estimate      object
Job Description      object
Company Name         object
Location             object
Headquarters         object
Size                 object
Founded               int64
Type of ownership    object
Industry             object
Sector               object
Revenue              object
dtype: object




Aucune de mes colonnes ne correspond à une date, je n'ai donc pas à faire de traitement


## 4. Traitement des doublons

### excel_table

In [15]:
print(df1.duplicated().value_counts())

False    5620
True       60
dtype: int64




60 lignes sont dupliquées.


In [16]:
df1.drop_duplicates()

,SalaryUSD,Country,PostalCode,EmploymentStatus,JobTitle,ManageStaff,YearsWithThisTypeOfJob,HowManyCompanies,OtherPeopleOnYourTeam,CompanyEmployeesOverall,PopulationOfLargestCityWithin20Miles,EmploymentSector,LookingForAnotherJob,CareerPlansThisYear,Gender,OtherJobDuties
0,115000.0,United States,03,Full time employee,DBA (Production Focus - build & troubleshoot s...,No,5,1 (this is the only company where I've had thi...,2,Not Asked,20K-99K (large town),Private business,Not Asked,"Stay with the same employer, same role",Male,Not Asked
1,100000.0,United States,None,Full time employee,"Developer: App code (C#, JS, etc)",No,6,1 (this is the only company where I've had thi...,More than 5,Not Asked,1M+ (metropolis),Private business,Not Asked,Prefer not to say,Male,Not Asked
3,70000.0,United States,94133,Full time employee,Analyst,No,3,1 (this is the only company where I've had thi...,2,Not Asked,1M+ (metropolis),Private business,Not Asked,Change both employers and roles,Male,Not Asked
4,110000.0,United States,95354,Full time employee,DBA (General - splits time evenly between writ...,No,25,3,None,Not Asked,100K-299K (city),Non-profit,Not Asked,"Stay with the same employer, same role",Male,Not Asked
5,79500.0,United States,78727,Full time employee,Analyst,No,2,1 (this is the only company where I've had thi...,None,Not Asked,1M+ (metropolis),Private business,Not Asked,"Stay with the same employer, same role",Male,Not Asked
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8609,66000.0,United States,Not Asked,Full time employee,"Developer: App code (C#, JS, etc)",No,10,Not Asked,None,Not Asked,Not Asked,Private business,"Yes, but only passively (just curious)",Not Asked,Not Asked,Not Asked
8611,96000.0,United States,Not Asked,Full time employee,DBA,No,5,Not Asked,None,Not Asked,Not Asked,Private business,"Yes, but only passively (just curious)",Not Asked,Not Asked,Not Asked
8612,85000.0,United States,Not Asked,Full time employee,DBA,No,2,Not Asked,None,Not Asked,Not Asked,Private business,"Yes, but only passively (just curious)",Not Asked,Not Asked,Not Asked
8614,137500.0,United States,Not Asked,Full time employee of a consulting/contracting...,DBA,No,6,Not Asked,4,Not Asked,Not Asked,Private business,No,Not Asked,Not Asked,Not Asked


On a supprimé les 60 lignes en double

### sql_table

In [17]:
print(dfk.duplicated().value_counts())

False    2253
dtype: int64




On est bon.


## 5. Regroupement des valeurs similaires


In [18]:
df1["JobTitle"].value_counts()

DBA (General - splits time evenly between writing & tuning queries AND building & troubleshooting servers)    1009
DBA                                                                                                            976
DBA (Production Focus - build & troubleshoot servers, HA/DR)                                                   583
Developer: T-SQL                                                                                               533
Architect                                                                                                      473
Manager                                                                                                        385
Developer: Business Intelligence (SSRS, PowerBI, etc)                                                          362
Analyst                                                                                                        352
Developer: App code (C#, JS, etc)                                               

On remarque que 4 catégories de 'JobTitle' commencent par DBA et 3 catégories commencent par Developer
Pour une meilleure lisibilité du diagramme circulaire, je les regroupe et renomme dans 'Database Administrator' et 'Developer'

In [19]:
df1["JobTitle"].str.startswith("DBA")

0        True
1       False
3       False
4        True
5       False
        ...  
8609    False
8611     True
8612     True
8614     True
8626     True
Name: JobTitle, Length: 5680, dtype: bool

In [20]:
df1.loc[df1["JobTitle"].str.startswith("DBA"), "JobTitle"] = "Database Administrator"
df1.loc[df1["JobTitle"].str.startswith("Developer"), "JobTitle"] = "Developer"

Il y a des catégories dans 'JobTitle' qui comportent moins de cinq entrées, je les regroupe dans une catégorie 'Other'

In [21]:
s = df1["JobTitle"].value_counts()
df1.loc[df1["JobTitle"].isin(s[(s < 5)].index), "JobTitle"] = "Other"

Je visualise la nouvelle distribution avec tous les éléments renommés et regroupés.

In [22]:
df1["JobTitle"].value_counts()

Database Administrator    2814
Developer                 1245
Architect                  473
Manager                    385
Analyst                    352
Engineer                   276
Other                       98
Data Scientist              37
Name: JobTitle, dtype: int64

In [23]:
df1.head()

,SalaryUSD,Country,PostalCode,EmploymentStatus,JobTitle,ManageStaff,YearsWithThisTypeOfJob,HowManyCompanies,OtherPeopleOnYourTeam,CompanyEmployeesOverall,PopulationOfLargestCityWithin20Miles,EmploymentSector,LookingForAnotherJob,CareerPlansThisYear,Gender,OtherJobDuties
0,115000.0,United States,03,Full time employee,Database Administrator,No,5,1 (this is the only company where I've had thi...,2,Not Asked,20K-99K (large town),Private business,Not Asked,"Stay with the same employer, same role",Male,Not Asked
1,100000.0,United States,None,Full time employee,Developer,No,6,1 (this is the only company where I've had thi...,More than 5,Not Asked,1M+ (metropolis),Private business,Not Asked,Prefer not to say,Male,Not Asked
3,70000.0,United States,94133,Full time employee,Analyst,No,3,1 (this is the only company where I've had thi...,2,Not Asked,1M+ (metropolis),Private business,Not Asked,Change both employers and roles,Male,Not Asked
4,110000.0,United States,95354,Full time employee,Database Administrator,No,25,3,None,Not Asked,100K-299K (city),Non-profit,Not Asked,"Stay with the same employer, same role",Male,Not Asked
5,79500.0,United States,78727,Full time employee,Analyst,No,2,1 (this is the only company where I've had thi...,None,Not Asked,1M+ (metropolis),Private business,Not Asked,"Stay with the same employer, same role",Male,Not Asked


Dans la colonne 'HowManyCompanies', je renomme les catégories en utilisant seulement un chiffre, sauf pour la réponse '6 or more'

In [24]:
df1["HowManyCompanies"].value_counts()

Not Asked                                                             3907
1 (this is the only company where I've had this kind of position)      621
2 (I worked at another similar position elsewhere before this one)     527
3                                                                      315
4                                                                      122
6 or more                                                              102
5                                                                       86
Name: HowManyCompanies, dtype: int64

In [25]:
df1.loc[df1["HowManyCompanies"] == "1 (this is the only company where I've had this kind of position)", "HowManyCompanies"] = "1"
df1.loc[df1["HowManyCompanies"] == "2 (I worked at another similar position elsewhere before this one)", "HowManyCompanies"] = "2"


Je les classe dans l'ordre croissant

In [26]:
df1["HowManyCompanies"].value_counts().sort_index()

1             621
2             527
3             315
4             122
5              86
6 or more     102
Not Asked    3907
Name: HowManyCompanies, dtype: int64

La même chose pour la colonne 'OtherPeopleOnYourTeam'

In [27]:
df1["OtherPeopleOnYourTeam"].value_counts().sort_index()

1              1158
2               645
3               495
4               323
5               208
More than 5     554
None           2297
Name: OtherPeopleOnYourTeam, dtype: int64

## 6. Sauvegarde dans MySQL

In [55]:
save_to_mysql(db_connect=connect,df_to_save=df1,df_name='excel_table_v1.0')

save_to_mysql(db_connect=connect,df_to_save=dfk,df_name='csv_table_v1.0')

print("Data saved to mysql")